In [9]:
import math
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import os
import pickle
import sklearn
import imblearn as imb
# print("imblearn version: ",imblearn.__version__)
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import itertools

from scipy.stats import randint

from rdkit import Chem
import sys
sys.path.append('../../../')
import utils
from VisUtils import *
from split_data import *
from RF_GSCV import *
# from RF_Utils import *


# Using pre-split files 
These are the original 'og' splits

In [10]:
split_path = '../../../../../data/NEK_data_4Berkeley/NEK2/'

train_x_df = pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_trainX.csv")
train_y_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_trainY.csv")
test_x_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_testX.csv")
test_y_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_testY.csv")
train_x = torch.from_numpy(train_x_df.to_numpy())
train_y = torch.from_numpy(train_y_df.to_numpy().reshape(-1))
test_x = torch.from_numpy(test_x_df.to_numpy())
test_y = torch.from_numpy(test_y_df.to_numpy().reshape(-1))
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

torch.Size([1635, 306]) torch.Size([1635]) torch.Size([409, 306]) torch.Size([409])


# Undersample

In [4]:

# # Scale data
x_df= pd.concat([train_x_df, test_x_df])
scaling=StandardScaler()
 
# Use fit and transform method 
scaling.fit(x_df)
Scaled_data=scaling.transform(x_df)
train_x_scaledtemp = scaling.transform(train_x_df)
test_x_scaledtemp = scaling.transform(test_x_df) 

undersample = RandomUnderSampler()
train_x_temp1, train_y_temp1 = undersample.fit_resample(train_x_scaledtemp, train_y_df)
train_x_UNDER = train_x_temp1
train_y_UNDER = train_y_temp1.to_numpy().flatten()
test_y_UNDER = test_y_df.to_numpy().flatten()
test_x_UNDER = test_x_scaledtemp

In [5]:
# # back to df for saving 
data_path = 'data/'
train_x_df_UNDER = pd.DataFrame(train_x_UNDER) 
train_y_df_UNDER = pd.DataFrame(train_y_UNDER) 
test_y_df_UNDER = pd.DataFrame(test_y_UNDER)
test_x_df_UNDER = pd.DataFrame(test_x_UNDER)

train_x_df_UNDER.to_csv(data_path+'inhib/inhib_train_x_UNDER.csv', index=False, header=False)
train_y_df_UNDER.to_csv(data_path+'inhib/inhib_train_y_UNDER.csv', index=False, header=False) 
test_y_df_UNDER.to_csv(data_path+'inhib/inhib_test_y_UNDER.csv', index=False, header=False) 
test_x_df_UNDER.to_csv(data_path+'inhib/inhib_test_x_UNDER.csv', index=False, header=False)

train_x_df_UNDER = pd.read_csv(split_path+'inhib/inhib_train_x_UNDER.csv')
train_y_df_UNDER= pd.read_csv(split_path+'inhib/inhib_train_y_UNDER.csv')
test_x_df_UNDER= pd.read_csv(split_path+'inhib/inhib_test_x_UNDER.csv')
test_y_df_UNDER= pd.read_csv(split_path+'inhib/inhib_test_y_UNDER.csv')

train_x_UNDER = train_x_df_UNDER.to_numpy()
train_y_UNDER = train_y_df_UNDER.to_numpy().reshape(-1)
test_x_UNDER = test_x_df_UNDER.to_numpy()
test_y_UNDER = test_y_df_UNDER.to_numpy().reshape(-1)
train_x_UNDER.shape, train_y_UNDER.shape, test_x_UNDER.shape, test_y_UNDER.shape


((223, 306), (223,), (408, 306), (408,))

# SMOTE 

In [6]:
# # Scale data
split_path = '../../../../../data/NEK_data_4Berkeley/NEK2/'
data_path = 'data/inhib/'

oversample = SMOTE() 
# train_x_temp2, train_y_temp2 = oversample.fit_resample(train_x_temp, train_y_df)
train_x_temp2, train_y_temp2 = oversample.fit_resample(train_x_scaledtemp, train_y_df)
train_x_SMOTE = train_x_temp2
train_y_SMOTE = train_y_temp2.to_numpy().flatten()
test_y_SMOTE = test_y_df.to_numpy().flatten()

# back to df for saving 
train_x_df_SMOTE = pd.DataFrame(train_x_SMOTE) 
train_y_df_SMOTE = pd.DataFrame(train_y_SMOTE) 
test_y_df_SMOTE = pd.DataFrame(test_y_SMOTE)
test_x_df_SMOTE = pd.DataFrame(test_x_scaledtemp)

train_x_df_SMOTE.to_csv(data_path+'inhib_train_x_SMOTE.csv', index=False, header=False)
train_y_df_SMOTE.to_csv(data_path+'inhib_train_y_SMOTE.csv', index=False, header=False) 
test_y_df_SMOTE.to_csv(data_path+'inhib_test_y_SMOTE.csv', index=False, header=False) 
test_x_df_SMOTE.to_csv(data_path+'inhib_test_x_SMOTE.csv', index=False, header=False) 
train_x_df_SMOTE = pd.read_csv(data_path+'inhib_train_x_SMOTE.csv')
train_y_df_SMOTE= pd.read_csv(data_path+'inhib_train_y_SMOTE.csv')
test_x_df_SMOTE= pd.read_csv(data_path+'inhib_test_x_SMOTE.csv')
test_y_df_SMOTE= pd.read_csv(data_path+'inhib_test_y_SMOTE.csv')

train_x_SMOTE= train_x_df_SMOTE.to_numpy()
train_y_SMOTE = train_y_df_SMOTE.to_numpy().reshape(-1)
test_x_SMOTE = test_x_df_SMOTE.to_numpy()
test_y_SMOTE = test_y_df_SMOTE.to_numpy().reshape(-1)
train_x_SMOTE.shape, train_y_SMOTE.shape, test_x_SMOTE.shape, test_y_SMOTE.shape

((3045, 306), (3045,), (408, 306), (408,))

# ADASYN

In [7]:
data_path = 'data/inhib/'
adasyn = ADASYN() 
train_x_temp3, train_y_temp3 = adasyn.fit_resample(train_x_scaledtemp, train_y_df)
train_x_ADASYN = train_x_temp3
train_y_ADASYN = train_y_temp3.to_numpy().flatten()
test_y_ADASYN = test_y_df.to_numpy().flatten()

# # back to df for saving 
train_x_df_ADASYN  = pd.DataFrame(train_x_ADASYN ) 
train_y_df_ADASYN  = pd.DataFrame(train_y_ADASYN ) 
test_y_df_ADASYN  = pd.DataFrame(test_y_ADASYN )
test_x_df_ADASYN  = pd.DataFrame(test_x_scaledtemp)

train_x_df_ADASYN .to_csv(data_path+'inhib_train_x_ADASYN.csv', index=False, header=False)
train_y_df_ADASYN .to_csv(data_path+'inhib_train_y_ADASYN.csv', index=False, header=False) 
test_y_df_ADASYN .to_csv(data_path+'inhib_test_y_ADASYN.csv', index=False, header=False) 
test_x_df_ADASYN .to_csv(data_path+'inhib_test_x_ADASYN.csv', index=False, header=False) 


train_x_df_ADASYN = pd.read_csv(data_path+'inhib_train_x_ADASYN.csv')
train_y_df_ADASYN= pd.read_csv(data_path+'inhib_train_y_ADASYN.csv')
test_x_df_ADASYN= pd.read_csv(data_path+'inhib_test_x_ADASYN.csv')
test_y_df_ADASYN= pd.read_csv(data_path+'inhib_test_y_ADASYN.csv')

train_x_ADASYN= train_x_df_ADASYN.to_numpy()
train_y_ADASYN = train_y_df_ADASYN.to_numpy().reshape(-1)
test_x_ADASYN = test_x_df_ADASYN.to_numpy()
test_y_ADASYN = test_y_df_ADASYN.to_numpy().reshape(-1)
train_x_ADASYN.shape, train_y_ADASYN.shape, test_x_ADASYN.shape, test_y_ADASYN.shape

((3036, 306), (3036,), (408, 306), (408,))

In [15]:
data_path = 'data/inhib/'
x_df = pd.concat([train_x_df, test_x_df])
scaling = StandardScaler()


scaling.fit(x_df)
train_x_scaledtemp = scaling.transform(train_x_df)
test_x_scaledtemp = scaling.transform(test_x_df)

train_x_scaled_df = pd.DataFrame(train_x_scaledtemp)
test_x_scaled_df = pd.DataFrame(test_x_scaledtemp)
train_y_df = pd.DataFrame(train_y_df).reset_index(drop=True) 
test_y_df = pd.DataFrame(test_y_df).reset_index(drop=True)   

train_x_scaled_df.to_csv(data_path + 'inhib_train_x_scaledoriginal.csv', index=False, header=False)
train_y_df.to_csv(data_path + 'inhib_train_y_scaledoriginal.csv', index=False, header=False)
test_x_scaled_df.to_csv(data_path + 'inhib_test_x_scaledoriginal.csv', index=False, header=False)
test_y_df.to_csv(data_path + 'inhib_test_y_scaledoriginal.csv', index=False, header=False)


train_x_scaled_df = pd.read_csv(data_path + 'inhib_train_x_scaledoriginal.csv', header=None)
train_y_df = pd.read_csv(data_path + 'inhib_train_y_scaledoriginal.csv', header=None)
test_x_scaled_df = pd.read_csv(data_path + 'inhib_test_x_scaledoriginal.csv', header=None)
test_y_df = pd.read_csv(data_path + 'inhib_test_y_scaledoriginal.csv', header=None)

train_x_scaled = train_x_scaled_df.to_numpy()
train_y = train_y_df.to_numpy().flatten()
test_x_scaled = test_x_scaled_df.to_numpy()
test_y = test_y_df.to_numpy().flatten()


print(train_x_scaled.shape, train_y.shape, test_x_scaled.shape, test_y.shape)

(1635, 306) (1635,) (409, 306) (409,)
